In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [2]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
train.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [4]:
test=pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")
print(test.head())

       id  age  gender   course  study_hours  class_attendance  \
0  630000   24   other       ba         6.85              65.2   
1  630001   18    male  diploma         6.61              45.0   
2  630002   24  female   b.tech         6.60              98.5   
3  630003   24    male  diploma         3.03              66.3   
4  630004   20  female   b.tech         2.03              42.4   

  internet_access  sleep_hours sleep_quality study_method facility_rating  \
0             yes          5.2          poor  group study            high   
1              no          9.3          poor     coaching             low   
2             yes          6.2          good  group study          medium   
3             yes          5.7       average        mixed          medium   
4             yes          9.2       average     coaching             low   

  exam_difficulty  
0            easy  
1            easy  
2        moderate  
3        moderate  
4        moderate  


In [5]:
num_col = ['age', 'study_hours', 'class_attendance', 'sleep_hours']

nominal_col = [
    'gender',
    'course',
    'internet_access',
    'study_method'
]

ordinal_col = [
    'sleep_quality',
    'facility_rating',
    'exam_difficulty'
]
target_col = 'exam_score'

In [6]:
print(train.head())

   id  age  gender   course  study_hours  class_attendance internet_access  \
0   0   21  female     b.sc         7.91              98.8              no   
1   1   18   other  diploma         4.95              94.8             yes   
2   2   20  female     b.sc         4.68              92.6             yes   
3   3   19    male     b.sc         2.00              49.5             yes   
4   4   23    male      bca         7.65              86.9             yes   

   sleep_hours sleep_quality   study_method facility_rating exam_difficulty  \
0          4.9       average  online videos             low            easy   
1          4.7          poor     self-study          medium        moderate   
2          5.8          poor       coaching            high        moderate   
3          8.3       average    group study            high        moderate   
4          9.6          good     self-study            high            easy   

   exam_score  
0        78.3  
1        46.7  
2       

In [7]:
# 'gender', 'course', 'internet_access'
test["gender"].unique()



array(['other', 'male', 'female'], dtype=object)

In [8]:
test["course"].unique()

array(['ba', 'diploma', 'b.tech', 'b.sc', 'b.com', 'bca', 'bba'],
      dtype=object)

In [9]:
nominal_col = [
    'gender',
    'course',
    'internet_access',
    'study_method'
]

In [10]:
# Ordinal categories (ORDER MATTERS)
# -------------------------
sleep_quality_order = ['poor', 'average', 'good']

facility_rating_order = ['low', 'medium', 'high']
exam_difficulty_order = ['easy', 'moderate', 'hard']


In [11]:
ordinal_encoder = OrdinalEncoder(categories=[
  sleep_quality_order,
  facility_rating_order,
  exam_difficulty_order
])

In [12]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_col),
        ('nominal', OneHotEncoder(handle_unknown='ignore'), nominal_col),
        ('ordinal', ordinal_encoder, ordinal_col)
    ]
)


In [13]:
# Model pipeline
# -------------------------
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(
        hidden_layer_sizes=(32, 16),
        max_iter=2000,
        random_state=42
    ))
])


In [14]:
# Train
# -------------------------
model.fit(train[num_col + nominal_col + ordinal_col], train[target_col])


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'study_hours',
                                                   'class_attendance',
                                                   'sleep_hours']),
                                                 ('nominal',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['gender', 'course',
                                                   'internet_access',
                                                   'study_method']),
                                                 ('ordinal',
                                                  OrdinalEncoder(categories=[['poor',
                                                                              'average',
                                                                              'good'],
                                                                             ['low',
                                                                              'medium',
                                                                              'high'],
                                                                             ['easy',
                                                                              'moderate',
                                                                              'hard']]),
                                                  ['sleep_quality',
                                                   'facility_rating',
                                                   'exam_difficulty'])])),
                ('regressor',
                 MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=2000,
                              random_state=42))])

In [15]:
# Predict
# -------------------------
exam_score_predictions = model.predict(
    test[num_col + nominal_col + ordinal_col]
)


In [16]:

# Submission
# -------------------------
output = pd.DataFrame({
    "id": test["id"],
    "exam_score": exam_score_predictions
})

output.to_csv("predictions.csv", index=False)
print("predictions.csv saved")

predictions.csv saved
